In [1]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git
!pip install datasets

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-bdjav7rr
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-bdjav7rr
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 3.3 MB 16.7 MB/s 
     |████████████████████████████████| 895 kB 20.7 MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-2.1.1-py3-none-any.whl size=2539209 sha256=3b4eb452df1cffe4c5636b3172555882335ad65eac7c354f9a06ae69f4ae9ff2
  Stored in directory: /tmp/pip-ephem-wheel-cache-w556z7ss/wheels/60/0e/d5/86e64fef3085f149b504f48af9ca6e8f6da427f42da9b4bbe5
Successfully built adapter-transformers
     |████████████████████████████████| 542 kB 11.4 MB/s 
     |████████████████████████████████| 118 kB 25.5 MB/s 
     |██████████████████████

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [4]:
import sys
sys.path.append('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/')

In [5]:
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"

In [6]:
path

'/content/gdrive/MyDrive/master_hpi/NLP_Project/code/'

In [16]:
from datasets import load_dataset

dataset = load_dataset("winogrande", "winogrande_s")
dataset.num_rows

#make dataset smaller for quicker training purpose



dataset= dataset.filter(lambda e, i: i<10000, with_indices=True)

Reusing dataset winogrande (/root/.cache/huggingface/datasets/winogrande/winogrande_s/1.1.0/b9e0e68fbf52c10bb774885e501dbb7407a239ce519cc8f62cea0025d48bc0ed)
Loading cached processed dataset at /root/.cache/huggingface/datasets/winogrande/winogrande_s/1.1.0/b9e0e68fbf52c10bb774885e501dbb7407a239ce519cc8f62cea0025d48bc0ed/cache-787ce0824ea2fb03.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/winogrande/winogrande_s/1.1.0/b9e0e68fbf52c10bb774885e501dbb7407a239ce519cc8f62cea0025d48bc0ed/cache-eb5bb9a0a4995bb5.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/winogrande/winogrande_s/1.1.0/b9e0e68fbf52c10bb774885e501dbb7407a239ce519cc8f62cea0025d48bc0ed/cache-f1630875c15e4abd.arrow


In [8]:
dataset['train'][0]
print(type(dataset['train'][0]['option2']))
print(type(dataset['test'][0]['sentence']))
print(type(dataset['train'][0]['answer']))



<class 'str'>
<class 'str'>
<class 'str'>


In [9]:
dataset['test'][0]


{'answer': '',
 'option1': 'Kenneth',
 'option2': 'Michael',
 'sentence': 'Kenneth went cheap on the gemstone present for Michael and _ was understanding about being a cheapskate.'}

In [10]:
dataset['validation'][0]


{'answer': '2',
 'option1': 'Sarah',
 'option2': 'Maria',
 'sentence': 'Sarah was a much better surgeon than Maria so _ always got the easier cases.'}

In [17]:
from transformers import BertTokenizer
from transformers import AutoConfig
from datasets import Value

#from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


#new_features = dataset["train"].features.copy()
#new_features["answer"] = Value('int64')
#dataset["train"] = dataset["train"].cast(new_features)
#dataset["train"].features

#new_features = dataset["validation"].features.copy()
#new_features["answer"] = Value('int64')
#dataset["validation"] = dataset["validation"].cast(new_features)
#dataset["validation"].features

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["option1"], max_length=1000, truncation=True, padding="max_length")

def encode_batch1(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["option2"], max_length=1000, truncation=True, padding="max_length")

def encode_batch2(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["sentence"], max_length=1000, truncation=True, padding="max_length")


# def xy(batch):
#   return tokenizer(batch["sentence"], max_length=1000, truncation=True, padding="max_length")




def answer_to_int(x):
    x['answer'] = 1 # + x['sentence1']
    return x

dataset = dataset.map(answer_to_int)

# dataset["train"]["answer"] = dataset["train"]["answer"].map(xy, batched=True)

dataset = dataset.map(encode_batch, batched=True)
dataset = dataset.map(encode_batch1, batched=True)
dataset= dataset.map(encode_batch2, batched=True)
print(dataset["train"].features)


dataset.set_format(type="torch", columns=["sentence", "option1", "option2", "answer"])
# dataset.set_format(type="torch", columns=['answer', 'attention_mask', 'input_ids', 'option1', 'option2', 'sentence', 'token_type_ids'])

print(dataset['train'])
#print(dataset['validation']['answer'])
print(dataset["train"][0])


print(dataset)


Loading cached processed dataset at /root/.cache/huggingface/datasets/winogrande/winogrande_s/1.1.0/b9e0e68fbf52c10bb774885e501dbb7407a239ce519cc8f62cea0025d48bc0ed/cache-e23b05f314800d06.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/winogrande/winogrande_s/1.1.0/b9e0e68fbf52c10bb774885e501dbb7407a239ce519cc8f62cea0025d48bc0ed/cache-04f7372304ffa923.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/winogrande/winogrande_s/1.1.0/b9e0e68fbf52c10bb774885e501dbb7407a239ce519cc8f62cea0025d48bc0ed/cache-b1ee50fa83d74da6.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

{'answer': Value(dtype='int64', id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'option1': Value(dtype='string', id=None), 'option2': Value(dtype='string', id=None), 'sentence': Value(dtype='string', id=None)}
Dataset({
    features: ['answer', 'attention_mask', 'input_ids', 'option1', 'option2', 'sentence'],
    num_rows: 640
})


TypeError: ignored

In [ ]:
print(dataset)


In [ ]:
#training
from transformers import AdapterConfig, AutoModelWithHeads
from transformers import BertTokenizer, BertForSequenceClassification

model = AutoModelWithHeads.from_pretrained("bert-base-uncased")
config = AdapterConfig.load("pfeiffer")
model.load_adapter("comsense/winogrande@ukp", config=config)



In [ ]:
# Add a new adapter
model.add_adapter("comsense/winogrande@ukp")

# Activate the adapter
model.train_adapter("comsense/winogrande@ukp")
model.set_active_adapters("comsense/winogrande@ukp")


In [ ]:
import numpy as np
from transformers import TrainingArguments, Trainer, EvalPrediction
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"
import os

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    remove_unused_columns=False,
    output_dir=path

)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
    adapter_names="winogrande",
    do_save_full_model=False,
    do_save_adapter_fusion=True
)



In [ ]:

trainer.train()



In [ ]:
trainer.evaluate()


In [ ]:
print(model.active_adapters)
model.save_adapter(path +"models/winogrande/", "winogrande")